In [22]:
import json
from itertools import chain
from pprint import pprint
from random import sample

import jsonlines

from hope.twitter.graph import get_graphing_info
from hope.twitter.item import TweetItem

In [2]:
DEBUG = False

node_set = set()
all_nodes = []
all_edges = []

In [3]:
with jsonlines.open("../data/2023Elections_sample.jsonl") as f:
    for line in f:
        _ = "" if line.get("_id", None) is None else line.pop("_id")
    
        nodes, edge = get_graphing_info(TweetItem(**line), tweets_as_nodes=True)
        if nodes:
            if DEBUG:
                pprint(nodes)
                pprint(edge)
            
            all_edges.append(edge)

            for node in nodes:
                if node["id"] not in node_set:
                    all_nodes.append(node)
                    node_set.add(node["id"])

In [4]:
len(all_edges)

293486

In [10]:
len(all_nodes)

357547

In [6]:
all_edges[0]

[{'source': '1626899713382383616',
  'type': 'Tweet-Directed',
  'action': 'retweet',
  'target': '1626882679995092992',
  'weight': 1}]

In [7]:
all_nodes[0]

{'id': '1626899713382383616',
 'tweet': 'RT @IAOkowa: We are counting down to #NigeriaDecides - only 6 days and 21 hours to go! \n\nToday, we are in Adamawa State, the birthplace of…',
 'user_id': '1590006005420756993',
 'timestamp': '2023-02-18T11:01:32Z'}

In [ ]:
actions = {"n_reply": 0, "n_retweet": 0, "n_quote": 0}  # Default values

In [8]:
node_mapping = {node["id"]: {**node, **actions} for node in all_nodes}

In [16]:
# Count number of retweets, replies, quotes for each node in sample
# TODO: Formalize into reusable function
for edge in chain(*all_edges):
    target_node: dict = node_mapping[edge["target"]]
    target_node[f"n_{edge['action']}"] += 1

In [28]:
idx = sample(range(len(all_nodes)), k=1)[0]     # Get random index
pprint(all_nodes[idx])

{'id': '1629059859311218693',
 'n_retweet': 1,
 'timestamp': '',
 'tweet': '',
 'user_id': '1482343842414309377'}


In [30]:
with open("../data/2023Elections_sample_nodes.jsonl", 'w') as f:
    for d in node_mapping.values():
        line = line = json.dumps(d) + "\n"
        f.write(line)

In [31]:
with open("../data/2023Elections_sample_edges.jsonl", "w") as f:
    for d in chain(*all_edges):
        line = line = json.dumps(d) + "\n"
        f.write(line)